In [5]:
# -*- coding: utf-8 -*-
import pandas as pd
import json

In [6]:
mst_edges = pd.read_excel("../static/data/xlsx/2008 MST Edges.xlsx")

In [7]:
skill_dna_2008_df = pd.read_excel("../static/data/xlsx/Skill DNA v2.xlsx", sheetname='2008')
skill_dna_2020_df = pd.read_excel("../static/data/xlsx/Skill DNA v2.xlsx", sheetname='2020')
skill_dna_2008_2020_df = pd.read_excel("../static/data/xlsx/Skill DNA v2.xlsx", sheetname='2008-2020')

In [8]:
skill_dna_name_refer = pd.read_excel("../static/data/xlsx/2008_cluster.xlsx")

In [9]:
skill_code_to_name = {r['ElementID']:r['node'] for i,r in skill_dna_name_refer.iterrows()}
skill_code_to_task = {r['ElementID']:r['Task'] for i,r in skill_dna_name_refer.iterrows()}

In [10]:
skill_dna_2008_2020_df.head()

,SOC Code,Job Title,2.A.2.b,1.A.2.c.2,1.A.1.c.2,4.C.1.a.2.h,4.C.2.a.1.e,4.A.4.b.4,4.C.2.c.1.c,4.A.1.b.2,...,2.B.1.d,4.C.2.b.1.f,4.C.1.c.2,2.A.2.a,4.A.3.b.6,4.C.3.d.3,4.C.3.a.2.a,1.A.4.b.3,4.A.4.a.6,1.A.2.a.3
0,00-0000,Total Average,-0.000133,-0.000392,0.000193,0.000049,-6.753497e-08,-0.000170,-0.000057,-0.000040,...,0.000373,-0.000236,-0.000152,0.000488,-0.000151,-0.000043,-0.000231,0.000368,0.000007,0.000566
1,11-1011,Chief Executives,0.005412,0.000000,0.000172,-0.002910,1.041907e-04,0.002757,0.000000,0.000000,...,-0.003488,0.001398,-0.004684,0.000913,0.000000,0.000000,-0.001909,0.000000,-0.007704,0.000000
2,11-1021,General and Operations Managers,0.008178,0.000000,0.002584,-0.002268,0.000000e+00,0.003263,0.000158,-0.001778,...,0.002533,0.000000,-0.005459,0.004913,0.001413,0.000000,-0.009182,-0.012011,-0.012674,0.000000
3,11-2011,Advertising and Promotions Managers,0.003992,0.000000,-0.002907,0.014087,0.000000e+00,-0.011660,0.000000,0.000000,...,0.001970,0.000000,-0.000934,0.015322,0.000000,0.000000,-0.006465,0.000000,0.003005,0.000000
4,11-2021,Marketing Managers,0.017440,0.000000,0.000978,-0.008439,0.000000e+00,-0.003316,0.000000,0.000000,...,0.010507,0.000000,0.004989,0.005832,0.000000,0.000000,-0.003344,0.000000,-0.001917,0.000000


In [11]:
def standardizer(df, col):
    df[col] -= df[col].min()
    df[col] /= df[col].max()
    
    #df[col] = (30**df[col]-1)/30
    
    #df[col] = [0.5 if value > 0.5 else value for value in df[col]]
    
    #df[col] = df[col]*200
    #df[col] = df[col]+10
    return df

In [12]:
def standardizer_diff(df, col):
    df[col] -= df[col].min()
    df[col] /= df[col].max()
    
    df[col] = (50**df[col]-1)/50
    
    #df[col] = [0.6 if value > 0.6 else value for value in df[col]]
    
    #df[col] = df[col]*200
    #df[col] = df[col]+10
    return df

In [13]:
for col in skill_dna_2008_df.columns.tolist()[2:]:
    skill_dna_2008_df = standardizer(skill_dna_2008_df, col)
    skill_dna_2020_df = standardizer(skill_dna_2020_df, col)
    skill_dna_2008_2020_df = standardizer_diff(skill_dna_2008_2020_df, col)    


In [14]:
skill_dna_2008_2020_df.head()

,SOC Code,Job Title,2.A.2.b,1.A.2.c.2,1.A.1.c.2,4.C.1.a.2.h,4.C.2.a.1.e,4.A.4.b.4,4.C.2.c.1.c,4.A.1.b.2,...,2.B.1.d,4.C.2.b.1.f,4.C.1.c.2,2.A.2.a,4.A.3.b.6,4.C.3.d.3,4.C.3.a.2.a,1.A.4.b.3,4.A.4.a.6,1.A.2.a.3
0,00-0000,Total Average,0.182134,0.294097,0.310537,0.110194,0.101783,0.105480,0.215812,0.212749,...,0.087899,0.133618,0.141486,0.230953,0.096831,0.080478,0.125833,0.101979,0.066281,0.081757
1,11-1011,Chief Executives,0.249847,0.299282,0.310250,0.086027,0.102379,0.134599,0.216714,0.213196,...,0.071319,0.149564,0.105337,0.236225,0.097778,0.080655,0.115064,0.098548,0.045431,0.077860
2,11-1021,General and Operations Managers,0.291689,0.299282,0.345261,0.090858,0.101784,0.140278,0.219236,0.194098,...,0.098457,0.135828,0.100025,0.291686,0.107019,0.080655,0.076839,0.026682,0.034746,0.077860
3,11-2011,Advertising and Promotions Managers,0.230600,0.299282,0.270371,0.324818,0.101784,0.035304,0.216714,0.213196,...,0.095610,0.135828,0.134576,0.498953,0.097778,0.080655,0.089657,0.098548,0.076078,0.077860
4,11-2021,Marketing Managers,0.485009,0.299282,0.321551,0.052249,0.101784,0.080267,0.216714,0.213196,...,0.147190,0.135828,0.195265,0.306027,0.097778,0.080655,0.106480,0.098548,0.060527,0.077860


# Skill Network Edges

In [15]:
mst_edges.to_csv("../static/data/network_csv/skill_edges_2008.csv", index=None)

# Skill Network Nodes

In [16]:
cols = skill_dna_2008_df.columns.tolist()[2:]

In [17]:
for i, r in skill_dna_2008_df.iterrows():
    skillmap = {}
    skillmap["job_title"] = r['Job Title']
    skillmap["job_code"] = r['SOC Code']  
    
    skill_list = []
    
    for col in cols:
        skill_dict = {"skill_name": skill_code_to_name[col],
                      "skill_code":col,
                      "task_name": skill_code_to_task[col],
                      "automation_2020":skill_dna_2020_df.iloc[i][col],
                      "automation_2008":skill_dna_2008_df.iloc[i][col],                                     
                      "automation_2008_2020":skill_dna_2008_2020_df.iloc[i][col],
                     }
        skill_list.append(skill_dict)  

    skillmap["skill_list"] = skill_list
        
    #with open('../static/data/network/'+r1['Job Code']+'-2020'+'.json', 'w') as outfile:
    with open('../static/data/network_2008/'+skill_dna_2008_df.iloc[i]['SOC Code']+'.json', 'w') as outfile:
        json.dump(skillmap, outfile)        

# Task Clusters

In [1]:
import pandas as pd

In [33]:
task_cluster = {}

task_cluster['Race with the Machine'] = ['Interpersonal Task','Group Task','Conflict Resolution','Management and Supervision']

task_cluster['Race against the Machine'] = ['Routine Problem Solving','Information Processing','General Physical Task','Dynamic Physical Task']

task_cluster['Race ahead of the Machine'] = ['Communication','Systems Analysis','Creative and Critical Thinking']

task_cluster['Running a Different Race'] = ['Equipment Maintenance','Vehicle and Equipment Operation','Machine Control','Hazardous Task']

In [34]:
flatten_task_cluster = []

for task_cluster_nm, tasks in task_cluster.items():
    for task in tasks:
        flatten_task_cluster_dict = {}
        flatten_task_cluster_dict['task_name'] = task
        flatten_task_cluster_dict['task_cluster'] = task_cluster_nm
        flatten_task_cluster.append(flatten_task_cluster_dict)

In [35]:
task_cluster_df = pd.DataFrame(flatten_task_cluster)

In [36]:
task_cluster_df.to_csv("../static/data/network_csv/task_cluster.csv", index=None)

# 2008 Cluster